In [94]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as pltd
import seaborn as sns
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [80]:
df_data = pd.read_csv('https://drive.google.com/uc?export=download&id=1tdnYgbK3-OJeeMK-QrW-ha6WgXcC79hE', low_memory=False, encoding='latin-1')
df_data_by_artist = pd.read_csv('https://drive.google.com/uc?export=download&id=1oPnKdpQLK7CO5rUVrMwvTvQuBGSmvRLH', low_memory=False, encoding='latin-1')
df_data_by_genres = pd.read_csv('https://drive.google.com/uc?export=download&id=12lqn5lD6x5Yh6mezlg6TQ60YA-Pn8aeV', low_memory=False, encoding='latin-1')
df_data_by_year = pd.read_csv('https://drive.google.com/uc?export=download&id=1zZrUYorbuT5MTCrzqvJDiK-W4M48IrM_', low_memory=False, encoding='latin-1')
df_data_w_genres = pd.read_csv('https://drive.google.com/uc?export=download&id=1fDUFtB5CG4MBnfj80q2h9r47Nlwzy9t1', low_memory=False, encoding='latin-1')

#Foi analisado que apenas o dataframe df_data possui os nomes das músicas. O que é necessário para a indicação
#Assim trabalharemos apenas com ele no momento

# Recomendação por features das músicas

In [82]:
df_data.head(2)

,valence,year,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo
0,0.0594,1921,0.982,"['Sergei Rachmaninoff', 'James Levine', 'Berli...",0.279,831667,0.211,0,4BJqT0PrAfrxzMOxytFOIz,0.878,10,0.665,-20.096,1,"Piano Concerto No. 3 in D Minor, Op. 30: III. ...",4,1921,0.0366,80.954
1,0.9630,1921,0.732,['Dennis Day'],0.819,180533,0.341,0,7xPhfUan2yNtyFG0cUWkt8,0.000,7,0.160,-12.441,1,Clancy Lowered the Boom,5,1921,0.4150,60.936


In [83]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170653 entries, 0 to 170652
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   valence           170653 non-null  float64
 1   year              170653 non-null  int64  
 2   acousticness      170653 non-null  float64
 3   artists           170653 non-null  object 
 4   danceability      170653 non-null  float64
 5   duration_ms       170653 non-null  int64  
 6   energy            170653 non-null  float64
 7   explicit          170653 non-null  int64  
 8   id                170653 non-null  object 
 9   instrumentalness  170653 non-null  float64
 10  key               170653 non-null  int64  
 11  liveness          170653 non-null  float64
 12  loudness          170653 non-null  float64
 13  mode              170653 non-null  int64  
 14  name              170653 non-null  object 
 15  popularity        170653 non-null  int64  
 16  release_date      17

In [85]:
valores_distintos = df_data.query('popularity == popularity')['popularity'].unique()
np.sort(valores_distintos)

#Aparentemente a coluna popularity é a popularidade das músicas, que varia de 0 a 100

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  99, 100])

In [89]:
#removendo duplicatas
name_artist = ['name','artists']
df_data = df_data.drop_duplicates(subset=name_artist,keep='last')

In [90]:
df = df_data[['acousticness','danceability','energy','instrumentalness', 'liveness', 'loudness', 'speechiness','valence']]

In [91]:
#função que retorna as músicas similares
def get_similarities(vetor):
    # vetor['acousticness','danceability','energy','instrumentalness', 'liveness', 'loudness', 'speechiness','valence']
    input_vector = vetor
    cosine_similarities = cosine_similarity(df, input_vector)
    df_retorno_genres = df.iloc[cosine_similarities[:,0].argsort()[::-1][:30]].merge(df_data[['id','name','artists','popularity']],how = 'inner',left_index=True,right_index=True).sort_values('popularity',ascending=False).head(10)
    return df_retorno_genres

In [95]:
#biblioteca picke para criar um pipeline
pickle.dump(get_similarities,open('similarity_01.pkl','wb'))

In [96]:
importar = pickle.load(open('similarity_01.pkl', 'rb'))

In [97]:
importar([[0.98200,	0.279,	0.211,	0.878000,	0.6650,	-20.096,	0.0366,	0.0594]])

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,id,name,artists,popularity
135875,0.921,0.283,0.0938,0.847,0.567,-19.184,0.0368,0.1470,4Fh72Bog0nlUUK8ygiBIg1,"Romeo and Juliet, Op. 64, Act I: Dance of the ...","['Sergei Prokofiev', 'Ukraine National Symphon...",37
133198,0.958,0.354,0.0719,0.821,0.558,-19.168,0.0432,0.1020,5m1wCJzZB5A3xdisjgncsC,On Golden Pond - Main Theme,['Dave Grusin'],26
113664,0.928,0.217,0.2610,0.798,0.714,-19.097,0.0890,0.0920,6nl4q6ukiqrliQt6larrph,The Flowermarket* - Instrumental,['AndrÃ© Previn'],21
1631,0.975,0.370,0.1890,0.842,0.694,-22.010,0.0442,0.0734,19Ol0MHJQ9aFXy1k1717Yq,"Mephisto Waltz No. 1, S. 514","['Franz Liszt', 'Vladimir Horowitz']",15
1629,0.981,0.430,0.2030,0.886,0.740,-19.760,0.0427,0.0688,6lDaWKLr7PtqT1iDZTDLaE,"Scherzo & March, S. 177","['Franz Liszt', 'Vladimir Horowitz']",15
1633,0.982,0.295,0.0713,0.891,0.699,-20.649,0.0472,0.0379,34y6Gie19NTeapGMoqCHhl,"AnnÃ©es de pÃ¨lerinage, PremiÃ¨re annÃ©e (Suis...","['Franz Liszt', 'Vladimir Horowitz']",14
24462,0.979,0.309,0.1090,0.865,0.855,-21.990,0.0380,0.0650,2vno1PfgJ7gcqJ9CSlzYsp,"Le tombeau de Couperin, M. 68a: I. PrÃ©lude","['Maurice Ravel', 'Charles Gerhardt', 'Nationa...",13
144138,0.880,0.331,0.2910,0.866,0.638,-19.403,0.0546,0.1650,6OJvW5OvXSnKBmNOL89Idk,Detour Ahead - Live / Take 1,['Bill Evans Trio'],12
1677,0.933,0.288,0.1450,0.760,0.717,-19.411,0.0359,0.0798,7paJusVgjTQGvFZpTDHYBW,"Sonata for Piano in B Minor, S. 178: III. Alle...","['Franz Liszt', 'Vladimir Horowitz']",9
4980,0.989,0.205,0.1860,0.759,0.555,-18.423,0.0338,0.1750,5UU8V2btFSjNZQsU0gxPFw,à¹à¸à¸¥à¹à¸£à¸¸à¹à¸,['University of North Texas Symphony Orchestra'],7
